In [8]:
# Y axis is elevation
from os import listdir
import numpy as np
import pykitti
import matplotlib.pyplot as plt
from scipy.misc import imread

In [2]:
def load_data():
    directory = "/home/sexy/Documents/dataset/processed_imgs/sequence"
    list_X = []
    for i in range(11):
        dirs = listdir(directory+str(i))
        dirs.sort()
        array = np.zeros((len(dirs), 256, 256, 3),dtype='uint8')
        for j, dirj in enumerate(dirs):
            array[j,:,:,:] = imread(directory+str(i)+"/"+dirj)
        list_X.append(array)
    return list_X

In [11]:
def load_poses():
    directoryL = "/home/sexy/Documents/dataset/"
    list_Y = []
    for i in range(11):
        data = pykitti.odometry(directoryL, str(i).zfill(2))
        data.load_poses()
        Y = np.zeros((len(data.T_w_cam0),3))
        for j in range(len(data.T_w_cam0)):
            Y[j,:] = data.T_w_cam0[j][:-1,-1]
        delta_Y = np.zeros((len(data.T_w_cam0),3))
        delta_Y[1:,:] = Y[1:,:]-Y[:-1,:]
        list_Y.append(delta_Y)
    print (list_Y[0:10][:])
    return list_Y

In [12]:
data = load_poses()

Loading poses for sequence 00...
done.
Loading poses for sequence 01...
done.
Loading poses for sequence 02...
done.
Loading poses for sequence 03...
done.
Loading poses for sequence 04...
done.
Loading poses for sequence 05...
done.
Loading poses for sequence 06...
done.
Loading poses for sequence 07...
done.
Loading poses for sequence 08...
done.
Loading poses for sequence 09...
done.
Loading poses for sequence 10...
done.
[array([[ 0.        ,  0.        ,  0.        ],
       [-0.04690294, -0.02839928,  0.8586941 ],
       [-0.04684051, -0.02836136,  0.8575809 ],
       ..., 
       [-0.051875  , -0.015615  ,  1.13321   ],
       [-0.053145  , -0.028569  ,  1.12672   ],
       [-0.059925  , -0.036026  ,  1.13392   ]]), array([[ 0.        ,  0.        ,  0.        ],
       [ 0.05154656, -0.02424883,  1.000725  ],
       [ 0.09491664, -0.02626739,  0.994934  ],
       ..., 
       [ 0.908     , -0.10739   ,  1.459     ],
       [ 0.878     , -0.10365   ,  1.406     ],
       [ 0.908

In [5]:
def knownEnv(data, poses):
    sequences = [0,1,2,3,4,5,6,7,8,9,10]
    ind = [] #80% lengths
    ind_total = [] #total lengths
    for i in sequences:
        ind_total.append(np.size(data[i],axis=0))
        ind.append(int(ind_total[i]*4/5.0))
    Xtr = np.zeros((sum(ind)-len(sequences),256,256,6), dtype="uint8") #sum of 80% lengths - first from 11
    Ytr = np.zeros((sum(ind)-len(sequences),3)) #same
    Xte = np.zeros((sum(ind_total)-sum(ind),256,256,6), dtype="uint8") #remainder of total minus 80% - first 11
    Yte = np.zeros((sum(ind_total)-sum(ind),3)) #same
    
    countTr = 0
    countTe = 0
    
    for i in sequences:
        Xtr[countTr:countTr+ind[i]-1,:,:,:3] = data[i][:ind[i]-1,:,:,:] #0 -> not inclusive 
        Xtr[countTr:countTr+ind[i]-1,:,:,3:] = data[i][1:ind[i],:,:,:]
        countTr += ind[i]-1
        
        #0 -> total-80%
        Xte[countTe:countTe+(ind_total[i]-ind[i]),:,:,:3] = data[i][(ind[i]-1):-1,:,:,:]
        Xte[countTe:countTe+(ind_total[i]-ind[i]),:,:,3:] = data[i][ind[i]:,:,:,:]
        countTe += (ind_total[i]-ind[i])
    
    for p in range(18541):
        if (np.sum(Xtr[p,:,:,5])==0 or np.sum(Xtr[p,:,:,1])==0):
            print ("You done messed up (don't swear!).")
    
    for q in range(4649):
        if (np.sum(Xte[q,:,:,5])==0 or np.sum(Xte[q,:,:,1])==0):
            print ("You done mest up (don't swear!).")
    
    print ("Yay! Maybe?")

In [6]:
knownEnv(data,load_poses())

Loading poses for sequence 00...
done.
Loading poses for sequence 01...
done.
Loading poses for sequence 02...
done.
Loading poses for sequence 03...
done.
Loading poses for sequence 04...
done.
Loading poses for sequence 05...
done.
Loading poses for sequence 06...
done.
Loading poses for sequence 07...
done.
Loading poses for sequence 08...
done.
Loading poses for sequence 09...
done.
Loading poses for sequence 10...
done.
Yay! Maybe?


In [7]:
print (18560+4640)

23200
